In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates
from matplotlib.collections import LineCollection
import plotly
import plotly.graph_objects as go

In [12]:
file = r'Pistachio_Low.csv'
df = pd.read_csv(file)
df['datetime'] = pd.to_datetime(df['DATE'])
#df
df2 = df.set_index(df.datetime)
#df2 = df2.drop(['datetime'], axis=1)
df2 = df2.drop(['DATE'], axis=1)

df2 = df2.sort_index() # For the monthly file, 1/1/16-7/1/16 were misplaced. Have to rank this for plotting

df_nan = df2[np.isnan(df2['ET'])]

# Use linear interpolation to simulate gap data that match the interval
filled = df2.interpolate(limit=2, method='linear')

df_filled = df_nan.merge(filled, left_index=True, right_index=True, how='left')
df_filled.drop(df_filled.columns[[0, 1, 2]], axis = 1, inplace = True) 

xmin = min(df2.index)
xmax = max(df2.index)

In [13]:
# Creating masks for periods with gaps 
s1mask = np.isfinite(df2.ET)
s2mask = np.isfinite(df2.ET_corr)

# Creating masks for models
SIMSmask = np.isfinite(df2.SIMS_3x3)
# DAmask = np.isfinite(df2.DISALEXI_3x3)
EEmask = np.isfinite(df2.EEMETRIC_3x3)
PTmask = np.isfinite(df2.PTJPL_3x3)
BOPmask = np.isfinite(df2.SSEBOP_3x3)
ENmask = np.isfinite(df2.ensemble_mean_3x3)

In [24]:
fig = go.Figure()

#          ----------------
# <<< --- |Adjustement Area| --- >>>
#          ----------------
# Trace 1 -> Unclosed flux 
fig.add_trace(go.Scatter(
                x=df2.index, 
                y=df2.ET,
                name = 'Unclosed ETa',
                fill= 'none',
                mode='lines',
                line_color='dimgrey',
                line_width=0, 
                showlegend=False))

# Trace 2 -> Closed flux for establish band
fig.add_trace(go.Scatter(
                x=df2.index, 
                y=df2.ET_corr,
                name = 'Energy Imbalance Band',
                fill= 'tonexty',
                mode='lines',
                line_color='dimgrey',
                line_width=0))

#          ---------------
# <<< --- |Model Variables| --- >>>
#          ---------------

# Trace 3 -> SIMS
fig.add_trace(go.Scatter(
                x=df2.index[SIMSmask],
                y=df2.SIMS_3x3[SIMSmask],
                name="SIMS_ETc",
                mode='lines+markers', 
                line_color='royalblue',
                opacity=0.8, 
                marker = dict(color='royalblue', size = 8, symbol = 0, line=dict(width=1,
                                        color='black'))))
# Trace 4 -> DISALEXI
# fig.add_trace(go.Scatter(
#                x=df2.index[DAmask],
#                y=df2.DISALEXI_3x3[DAmask],
#                name="DisALEXI_ETc",
#                mode='markers', #'lines+markers'
#                #line_color='BurlyWood',
#                opacity=0.8,
#                marker = dict(color='BurlyWood', size = 8, symbol = 2, line=dict(width=1,
#                                        color='black'))))

# Trace 5 -> EEMETRIC
fig.add_trace(go.Scatter(
                x=df2.index[EEmask],
                y=df2.EEMETRIC_3x3[EEmask],
                name="EEMETRIC_ETc",
                mode='lines+markers', 
                line_color='coral',
                opacity=0.8,
                marker = dict(color='coral', size = 8, symbol = 1, line=dict(width=1,
                                        color='black'))))
# Trace 6 -> PTJPL
fig.add_trace(go.Scatter(
                x=df2.index[PTmask],
                y=df2.PTJPL_3x3[PTmask],
                name="PTJPL_ETc",
                mode='lines+markers', 
                line_color='MediumSpringGreen',
                opacity=0.8,
                marker = dict(color='MediumSpringGreen', size = 8, symbol = 5, line=dict(width=1,
                                        color='black'))))
# Trace 7 -> SSEBOP
fig.add_trace(go.Scatter(
                x=df2.index[BOPmask],
                y=df2.SSEBOP_3x3[BOPmask],
                name="SSEBOP_ETc",
                mode='lines+markers', 
                line_color='orchid',
                opacity=0.8, 
                marker = dict(color='orchid', size = 8, symbol = 14, line=dict(width=1,
                                        color='black'))))
# Trace 8 -> Ensemble
fig.add_trace(go.Scatter(
                x=df2.index[ENmask],
                y=df2.ensemble_mean_3x3[ENmask],
                name="Ensemble_ETc",
                mode='lines+markers', 
                line_color='red',
                opacity=0.7, 
                marker = dict(color='red', size = 9, symbol = 17, line=dict(width=1,
                                        color='black'))))


#           ---------
# <<< ---- |Flux Data| ---- >>>
#           ---------

# Trace 9 -> Unclosed flux for reference
fig.add_trace(go.Scatter(
    x=df2.index, 
    y=df2.ET,
    name = 'Unclosed ETa',
    mode='none',
    showlegend=False,
    marker=dict(color='rgba(0, 0, 0, 0)',
            size=4,
            line=dict(color='red',width=1))))

# Trace 10 -> Closed flux for reference 
fig.add_trace(go.Scatter(
    x=df2.index, 
    y=df2.ET_corr,
    name = 'Closed ETa',
    mode='none',
    showlegend=False,
    marker=dict(color='rgba(0, 0, 0, 0)',
            size=4,
            line=dict(color='black',width=1))))

#fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='darkgray')

fig.update_layout(
    autosize=True,
    title={'text':"Intercomparson of Monthly ET (mm) Pistachio Low Salinity [3x3 Grid] <br> EBC = (H+LE)/(Rn-G) = 0.71; Correction Factor = 1/EBC = 1.41",
          'x':0.145,
          'y':0.93,
          'xanchor': 'left',
          'yanchor': 'top'},
    xaxis_title= "Date Slider",
    yaxis_title= "Evapotranspiration (mm)",              
    xaxis_rangeslider_visible=True,
    plot_bgcolor='rgba(0,0,0,0)',
    titlefont=dict(
        family="Times New Roman",
        size=22,
        color='black'),    
    font=dict(
        family="Times New Roman",
        size=18,
        color='black'))

# Add Dropdown Menu Option
# For visible: all traces are ranked sequentially as listed above (e.g. Trace#)

fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=list([
                dict(label="All",
                     method="update",
                     args=[{"visible": [True]},
                           {"title": "Intercomparson of Monthly ET (mm) Pistachio Low Salinity [3x3 Grid] <br> EBC = (H+LE)/(Rn-G) = 0.71; Correction Factor = 1/EBC = 1.41",
                            }]),
                dict(label="SIMS",
                     method="update",
                     args=[{"visible": [True, True, True, False, False, False, True, True, True]},
                           {"title": "SIMS Monthly Evapotranspiration (mm)",
                            }]),
                dict(label="EEMETRIC",
                     method="update",
                     args=[{"visible": [True, True, False, True, False, False, True, True, True]},
                           {"title": "EEMETRIC Monthly Evapotranspiration (mm)",
                            }]),
                dict(label="PT-JPL",
                     method="update",
                     args=[{"visible": [True, True, False, False, True, False, True, True, True]},
                           {"title": "PT-JPL Monthly Evapotranspiration (mm)",
                            }]),
                dict(label="SSEBop",
                     method="update",
                     args=[{"visible": [True, True, False, False, False, True, True, True, True]},
                           {"title": "SSEBop Monthly Evapotranspiration (mm)",
                            }]),
                dict(label="Ensemble",
                     method="update",
                     args=[{"visible": [True, True, False, False, False, False, True, True, True]},
                           {"title": "Ensemble Monthly Evapotranspiration (mm)",
                            }]),
                        ]),
                direction="down",
                    pad={"r": 10, "t": 10},
                    showactive=True,
                    x=-0.23,
                    xanchor="left",
                    y=1.08,
                    yanchor="top")],
                annotations=[
                    dict(text="<b>Traces</b>", x=-0.23, xref="paper", y=1.12, yref="paper",
                             align="left", showarrow=False)])

fig.update_xaxes(showline=True, 
                 linewidth=1, 
                 linecolor='black', 
                 mirror=True) 
                 #showgrid=True, 
                 #gridwidth=1, 
                 #gridcolor='Gainsboro')
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)

fig.show(config={'scrollZoom': True})

In [25]:
# The plot above isn't well sized, when exporting, the .html will autosize
plotly.offline.plot(fig, config={'scrollZoom': True}, filename = 'Monthly_Pist_Low_INT_3x3.html', auto_open=False)

'Monthly_Pist_Low_INT_3x3.html'